In [9]:
import cv2
import mediapipe as mp
from numpy import math
import numpy as np
import time


mpPose=mp.solutions.pose
pose = mpPose.Pose()  

video = cv2.VideoCapture(r'C:\Users\siddharth\Dropbox\PC\Downloads\KneeBendVideo.mp4')

if (video.isOpened() == False): 
    print("Error")
    

frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)
result = cv2.VideoWriter(r'C:\Users\siddharth\Dropbox\PC\Downloads\output.mp4', cv2.VideoWriter_fourcc(*'MP4V'),20, size)


up= False #flag if the knee is bend or flat
counter =0  # for counting the reps
t=int(8)  # timer
timer=False  # flag to start the timer
fail=False   # flag for a successful rep

    
while(True):
    angles = []    # list for calculating the angles
    LMlist = []    # list for making the landmarks
    ret, img = video.read()
  
    if ret == True: 
  
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    # converting it in RGB format
        res = pose.process(rgb)
        
        if res.pose_landmarks is not None:
            for id, lm in enumerate(res.pose_landmarks.landmark):
                h, w, c = img.shape    # getting the shape of the frame
                cx, cy = int(lm.x * w), int(lm.y * h)   #getting the pixel coordinate for each landmark
                LMlist.append([id, cx, cy])    #appending it into the list
                
                
            a= np.array(LMlist[23][1:]) #landmark of left hip
            b= np.array(LMlist[25][1:]) #landmark of left knee
            c= np.array(LMlist[27][1:]) #landmark of left ankel
            
            radians = np.arctan2(c[1] - b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0]) #calculating the angles
            angle = np.abs(radians*180.0/np.pi)  #converting radian into degree
            
            
            if angle > 180.0:   #if obtuse
                angle = 360-angle
                
                
            cv2.putText(img, f'Angle:{int(angle)}', (50, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            cv2.line(img, (a[0], a[1]), (b[0], b[1]), (255, 255, 0), 2)
            cv2.line(img, (b[0], b[1]), (c[0], c[1]), (255, 255, 0), 2)
            cv2.circle(img, (a[0], a[1]), 5, (0, 0, 0), cv2.FILLED)
            cv2.circle(img, (b[0], b[1]), 5, (0, 0, 0), cv2.FILLED)
            cv2.circle(img, (c[0], c[1]), 5, (0, 0, 0), cv2.FILLED)
            
            
            if not up and (angle<140) : #if the knee previously was down and now is up
                up=True
                timer=True
                
                
            elif(angle>140): # if knee gets down
                prev = time.time()
                if 0<t<7:
                    fail=True   
                    
                elif fail:
                    cv2.putText(img, 'Please hold your leg up until the timer goes to 0', (50,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
                up =False
                t=int(8)
                
                
            elif timer: #if timer flag is true
                cur = time.time()
                
                if cur-prev >= 1:
                    prev = cur
                    t = t-1

                
                if t==0: # for a successful rep
                    counter+=1
                    timer= False
                    fail=False
                    
                elif fail:
                    cv2.putText(img, 'Please hold your leg up until the timer goes to 0', (50,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
                    
                else:
                    cv2.putText(img, 'Nice hold your leg up until the timer goes to 0', (50,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
            
            
            elif not fail and up:
                cv2.putText(img, 'Great rep is completed you can put your leg down', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)    
        
        
        cv2.putText(img, "REPS:"+str(counter), (50, 100), cv2.FONT_ITALIC, 1, (0, 255, 0), 2)
        cv2.putText(img, 'TIMER:'+str(t), (50, 130), cv2.FONT_ITALIC, 1, (255, 0, 0), 2)

        cv2.imshow('REP COUNTER', img)
        result.write(img)
  
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 
            
    else:
        break
 
video.release()
result.release()
cv2.destroyAllWindows()